# Bar in NYC (Manhattan, Brooklyn), Richmond VA, Seattle
Film production houses, film rental houses

Beer, fresh out of college drinks
- microbrew
- need fine options on food menu
- shitty vodka is best vodka

- microwinery <3 

_Target customers_<br>
- Middle-aged
- Mid-20s hipsters

Restaurant
SPORTS BAR that is always closed




Add location to a map through the import we learned on learn.co

# USE BASE DATA

First step is to get requests:

In [109]:
import requests
import json
import sys
import pandas as pd
import csv

print(sys.path)

['/Users/Mike/Flatiron_020121/yelp_phase1_project', '/opt/anaconda3/lib/python38.zip', '/opt/anaconda3/lib/python3.8', '/opt/anaconda3/lib/python3.8/lib-dynload', '', '/opt/anaconda3/lib/python3.8/site-packages', '/opt/anaconda3/lib/python3.8/site-packages/aeosa', '/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions', '/Users/Mike/.ipython']


In [110]:
url =  'https://api.yelp.com/v3/businesses/search' #points to url of user's yelp developer page

In [111]:
from  keys  import  client_id, api_key

In [112]:
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }


In [113]:
### THIS CAN BE DYNAMIC
# what type of business do you want to search
term = 'Film Production' # this would 
# term = 'Bar'
#where do you want to perform this search
location = 'Brooklyn'

# what is your other parameter you want to search against
categories = 'Video/Film'
# categories = 'Restaurant'

In [208]:

url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "categories" : categories,
                "limit": 50,
                "offset": 0
            }

In [162]:
response = requests.get(url, headers=headers, params=url_params)
json.loads(response.text)
response.json()
data = response.json()
print(data['total'], data.keys())
# print(response)

6200 dict_keys(['businesses', 'total', 'region'])


In [229]:
url_params

{'term': 'Bar',
 'location': 'Brooklyn',
 'categories': 'Restaurant',
 'limit': 50,
 'offset': 0}

In [230]:
def yelp_call(headers, url_params): 
    response = requests.get(url, headers=headers, params=url_params) # our url, header and params should be consistent, atleast with our Yelp data
    return response.json()

#### Parsing the Data

In [231]:
def parse_data(list_of_data):
    businesses = []
    for business in list_of_data:
        biz_price = None
        if 'price' not in business.keys():
            biz_price = None
        else:
            biz_price = len(business['price'])
        biz_tuple = (business['name'],
                     business['location']['display_address'],
                     business['location']['city'],
                     business['rating'],
                     business['review_count'],
                     business['coordinates'],
                     biz_price)
        businesses.append(biz_tuple)
    return businesses

    # create a container to hold our parsed data
    # loop through our business and 
    # parse each individual business into a tuple
    # add each individual business tuple to our data container
    # return the container with all of the parsed results



## Make a function that adds a price column, and if the dict doesnt have a price column, add a vlaue of Null

In [169]:
parsed_results # list of tuples representing name, address, and rating of establishments

[('The Rogers Garden',
  ['708 Rogers Ave', 'Brooklyn, NY 11226'],
  'Brooklyn',
  4.5,
  46,
  {'latitude': 40.65424833, 'longitude': -73.952965},
  None),
 ('Flatbush Zombie House',
  ['734 Flatbush Ave', 'Brooklyn, NY 11226'],
  'Brooklyn',
  4.0,
  102,
  {'latitude': 40.6550392260637, 'longitude': -73.9600387215614},
  2),
 ('End Zone Sports Bar & Lounge',
  ['28 Dooley St', 'Brooklyn, NY 11235'],
  'Brooklyn',
  5.0,
  6,
  {'latitude': 40.58437, 'longitude': -73.9453},
  None),
 ('Boobie Trap',
  ['308 Bleecker St', 'Brooklyn, NY 11237'],
  'Brooklyn',
  4.0,
  273,
  {'latitude': 40.7003517876193, 'longitude': -73.9161294247438},
  1),
 ('The Crown Inn',
  ['724 Franklin Ave', 'Brooklyn, NY 11238'],
  'Brooklyn',
  4.0,
  200,
  {'latitude': 40.673874, 'longitude': -73.957101},
  2),
 ('MOON BAR Rooftop',
  ['137-72 Northern Blvd', 'Fl 9', 'Flushing, NY 11354'],
  'Flushing',
  4.0,
  27,
  {'latitude': 40.7635, 'longitude': -73.82844},
  None),
 ('The Wilky',
  ['108 Patchen A

In [170]:
df = pd.DataFrame(parsed_results, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])

In [171]:
type(df)

pandas.core.frame.DataFrame

In [188]:
# your code to open the csv file, concat the current data, and save the data. 

def df_save(csv_filepath, df): # MIKE NOTE: an if statement for if the file exists may be useful
    with open(csv_filepath, "a") as f: #'x' creates and writes to csv
        read_file = csv.writer(f)
        df.to_csv(csv_filepath, mode = "a", index = False)
# this will append to csv. 2nd arg index=False to not save the inde

In [190]:
df_save(f'database/{term}_{location}_database.csv', df)

In [191]:
len(pd.read_csv(f'database/{term}_{location}_database.csv'))+1

153

In [192]:
num_test = len(pd.read_csv(f'database/{term}_{location}_database.csv'))
num_test + 1

153

In [214]:
data['total']

6200

In [228]:
len(yelp_call(0, headers, url_params)['businesses'])

50

In [252]:
def call_1000(csv_filepath):
    cur = 0
    num = data['total']
    #I think there is an issue with Yelp and their total values.  I think we could do some work around iwth an if statemnt that checks to see if there is a key called businesses, if not, end the loop
    while cur < (num) and cur < 20: #### MIKE NOTE I believe num should be referring to the number of results we have in our dataframe
        results = yelp_call(headers, url_params)
        parsed_results = parse_data(results['businesses'])
        df = pd.DataFrame(parsed_results, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
        df_save(csv_filepath, df)
        pd.read_csv(csv_filepath)
        cur += 50
        url_params['offset'] = cur
        if cur > 20:
            break

        
        #set up a while loop to go through and grab the result 
        #set the offset parameter to be where you currently are in the results 
        #make your API call with the new offset number
        #after you get your results you can now use your function to parse those results
        # use your function to insert your parsed results into the db
        #increment the counter by 50 to move on to the next results
    return pd.DataFrame(pd.read_csv(csv_filepath))

In [253]:
df = call_1000(f'database/{term}_{location}_database.csv')

In [256]:
df.tail()

,Name,Address,City,Rating,Review Count,Coordinates,Price
45,Friends and Lovers,"['641 Classon Ave', 'Brooklyn, NY 11238']",Brooklyn,4.0,77,"{'latitude': 40.678536, 'longitude': -73.958437}",2.0
46,Father Knows Best,"['611A Wilson Ave', 'Brooklyn, NY 11207']",Brooklyn,4.0,327,"{'latitude': 40.6901735, 'longitude': -73.9069...",2.0
47,Glorietta Baldy,"['502 Franklin Ave', 'Brooklyn, NY 11238']",Brooklyn,4.0,64,"{'latitude': 40.6813358395142, 'longitude': -7...",2.0
48,The Holler,"['348 Franklin Ave', 'Brooklyn, NY 11238']",Brooklyn,4.5,32,"{'latitude': 40.687208, 'longitude': -73.957163}",2.0
49,Parkside,"['705 Flatbush Ave', 'Brooklyn, NY 11225']",Brooklyn,4.0,186,"{'latitude': 40.655949, 'longitude': -73.959599}",2.0


In [244]:
len(df['Name'].unique())

201

In [130]:
#pd.DataFrame(parsed_results)

In [131]:
df.tail()

,Unnamed: 0,Name,Address,City,Rating,Review Count,Coordinates,Price
96,45.0,Bar Meridian,"['406 Prospect Pl', 'Brooklyn, NY 11238']",Brooklyn,4.5,11,"{'latitude': 40.6763427218772, 'longitude': -7...",NaN
97,46.0,Bodeguita,"['6 Suydam St', 'Brooklyn, NY 11221']",Brooklyn,4.5,38,"{'latitude': 40.69555, 'longitude': -73.93204}",NaN
98,47.0,Friends and Lovers,"['641 Classon Ave', 'Brooklyn, NY 11238']",Brooklyn,4.0,77,"{'latitude': 40.678536, 'longitude': -73.958437}",2.0
99,48.0,Fancy Nancy,"['1038 Bedford Ave', 'Brooklyn, NY 11205']",Brooklyn,4.0,166,"{'latitude': 40.6895502, 'longitude': -73.9554...",2.0
100,49.0,Father Knows Best,"['611A Wilson Ave', 'Brooklyn, NY 11207']",Brooklyn,4.0,327,"{'latitude': 40.6901735, 'longitude': -73.9069...",2.0


In [132]:
df.groupby('City')['Name'].count()

City
Brooklyn                     91
Carroll Gardens               2
City                          1
Downtown Brooklyn             1
Far Rockaway                  2
Flushing                      2
Prospect Lefferts Gardens     2
Name: Name, dtype: int64

In [133]:
'''
Arverne              1
Astoria              2
Bronx                1
Brooklyn            14
Hoboken              1
Jersey City          1
Long Island City     4
Middle Village       1
New York            20
Queens               5
'''

'\nArverne              1\nAstoria              2\nBronx                1\nBrooklyn            14\nHoboken              1\nJersey City          1\nLong Island City     4\nMiddle Village       1\nNew York            20\nQueens               5\n'

# Below are cells used to test work above and assignment outputs

### Organized by appropriate section:

#### Use Base Data

In [134]:
# print(api_key)

In [135]:
url_params

{'term': 'Bar',
 'location': 'Brooklyn',
 'categories': 'Restaurant',
 'limit': 50,
 'offset': 950}

In [136]:
len(yelp_call(0,headers,url_params)['businesses'])

50

(below will actually be more appropriate at bottom)

In [137]:
# def call_1000(): ## copied below in a more appropriate space
    
#     cur = 0
#     num = 1000
#     #set up a while loop to go through and grab the result 
#     while cur < num and cur < 1000: #### MIKE NOTE I believe num should be referring to the number of results we have in our dataframe
#         #set the offset parameter to be where you currently are in the results 
#         url_params['offset'] = cur
#         #make your API call with the new offset number
#         results = yelp_call(num, headers, url_params)

#         #after you get your results you can now use your function to parse those results
#         parsed_results = parse_results(results)

#         # use your function to insert your parsed results into the db
#         df_save(parsed_results)
#         #increment the counter by 50 to move on to the next results
#         cur += 50
#     return None

In [138]:
# call_1000()

#### Parse Data

In [139]:
# cur = 0

# #set up a while loop to go through and grab the result 
# while cur < num and cur < 1000:
#     #set the offset parameter to be where you currently are in the results 
#     url_params['offset'] = cur
#     #make your API call with the new offset number
#     results = yelp_call(url_params, api_key)
    
#     #after you get your results you can now use your function to parse those results
#     parsed_results = parse_results(results)
    
#     # use your function to insert your parsed results into the db
#     df_save(parsed_results)
#     #increment the counter by 50 to move on to the next results
#     cur += 50

In [140]:
# business = data['businesses'][44]

In [141]:

# name, rating, review_count, price, ##location?

In [142]:
# business['price'] # note: not every business has a 'price' key listed. this WOULD work if ran after parse_data() fn

In [143]:
# business['location']['city']

'Brooklyn'

In [144]:
# full_address = business['location']['address1'], business['location']['city'], business['location']['state'], business['location']['zip_code']
# full_address

('1237 Flushing Ave', 'Brooklyn', 'NY', '11237')

In [145]:
# business_rating = business['rating']
# business_rating

4.0

In [146]:
# biz_tuple = (business['name'], business['location']['display_address'], business['rating'])

In [147]:
# biz_tuple

('Jupiter Disco', ['1237 Flushing Ave', 'Brooklyn, NY 11237'], 4.0)